# **Deep Learning with SkyDL**
**Setup software libraries**

In [1]:
from SkyDL import skydl
import env

## **Training of Deep Learning models**
**Create `Trainer` object**

In [ ]:
SkyTrainer = skydl.Trainer(privatekey_path = env.privatekey_path)

**Get token**

In [ ]:
SkyTrainer.get_token(email='iker.sanchez@vizzuality.com')

## Database

**Retrive table names of the database**

In [ ]:
SkyTrainer.table_names

**Retrieve table from database**

In [ ]:
SkyTrainer.datasets

## Skydipper datasets for Deep Learning

In [ ]:
#SkyTrainer.datasets_api 

## Image composites

In [ ]:
SkyTrainer.composite(slugs=['Sentinel-2-Top-of-Atmosphere-Reflectance', 'Lake-Water-Quality-100m'],\
              init_date = '2019-01-21', end_date = '2019-01-31', zoom=6)

## Creation of Geostore

We select the areas from which we will export the training data.

In [ ]:
train_atts = {"type":"FeatureCollection",
              "features":[
                  {"type":"Feature",
                   "properties":{},
                   "geometry":{
                       "type":"Polygon",
                       "coordinates":[[[-0.406494140625,38.64476310916202],
                                       [0.27740478515625,38.64476310916202],
                                       [0.27740478515625,39.74521015328692],
                                       [-0.406494140625,39.74521015328692],
                                       [-0.406494140625,38.64476310916202]]]
                   }
                  },
                  {"type":"Feature",
                   "properties":{},
                   "geometry":{
                       "type":"Polygon",
                       "coordinates":[[[-1.70013427734375,35.15135442846945],
                                       [-0.703125,35.15135442846945],
                                       [-0.703125,35.94688293218141],
                                       [-1.70013427734375,35.94688293218141],
                                       [-1.70013427734375,35.15135442846945]]]
                   }
                  }
              ]
             }

train_atts = {"type":"FeatureCollection",
              "features":[
                  {"type":"Feature",
                   "properties":{},
                   "geometry":{
                       "type":"Polygon",
                       "coordinates":[[[-0.406494140625,38.64476310916202],
                                       [0.28,38.64476310916202],
                                       [0.28,39.74521015328692],
                                       [-0.406494140625,39.74521015328692],
                                       [-0.406494140625,38.64476310916202]]]
                   }
                  },
                  {"type":"Feature",
                   "properties":{},
                   "geometry":{
                       "type":"Polygon",
                       "coordinates":[[[-1.70013427734375,35.15135442846945],
                                       [-0.703125,35.15135442846945],
                                       [-0.703125,35.94688293218141],
                                       [-1.70013427734375,35.94688293218141],
                                       [-1.70013427734375,35.15135442846945]]]
                   }
                  }
              ]
             }

valid_atts = None
test_atts = None

In [ ]:
SkyTrainer.create_geostore_from_geojson(attributes=[train_atts, valid_atts, test_atts])

Check geostore object

In [ ]:
SkyTrainer.geostore

In [ ]:
SkyTrainer.nPolygons

Check geostore object on a server and display it on map

In [ ]:
#SkyTrainer.multipolygon

In [ ]:
#SkyTrainer.multipolygon.map()

## Data pre-processing

We normalize the composite images to have values from 0 to 1.

In [ ]:
SkyTrainer.normalize_images(scale=100, norm_type='geostore')

### Select input/output bands

In [ ]:
SkyTrainer.select_bands(input_bands = ['B2','B3','B4','B5','ndvi','ndwi'],\
                 output_bands = ['turbidity_blended_mean'])

In [ ]:
SkyTrainer.images

## Create TFRecords for training

In [ ]:
SkyTrainer.export_TFRecords(sample_size = 20000, kernel_size = 1)

In [ ]:
SkyTrainer.versions

## Training the model in AI Platform

In [ ]:
SkyTrainer.train_model_ai_platform(model_type='MLP', model_output='regression', model_architecture='sequential1',\
                                   model_name='water_quality', batch_size=4)

In [ ]:
SkyTrainer.training_params

In [ ]:
SkyTrainer.versions

## Deployed the model to AI Platform

Before it's possible to get predictions from the trained model, it needs to be deployed on AI Platform.  The first step is to create the model.  The second step is to create a version.  See [this guide](https://cloud.google.com/ml-engine/docs/tensorflow/deploying-models) for details.  Note that models and versions can be monitored from the [AI Platform models page](http://console.cloud.google.com/ai-platform/models) of the Cloud Console. 

In [ ]:
SkyTrainer.deploy_model_ai_platform()

In [ ]:
SkyTrainer.versions

***
## **Validation of Deep Learning models**
**Create `Validator` object**

In [2]:
SkyValidator = skydl.Validator()

### Select model

In [3]:
SkyValidator.models

,model_name,model_type,model_output,model_description,output_image_id
0,water_quality,MLP,regression,,1


In [4]:
versions = SkyValidator.select_model(model_name='water_quality')

The water_quality model has the following versions: [1581938271]


### Select version

In [6]:
SkyValidator.select_version(version=versions[0])

Selected version name: v1581938271
Datasets:  ['Sentinel-2-Top-of-Atmosphere-Reflectance', 'Lake-Water-Quality-100m']
Bands:  [['B2', 'B3', 'B4', 'B5', 'ndvi', 'ndwi'], ['turbidity_blended_mean']]
scale:  100.0
init_date:  2019-01-21
end_date:  2019-01-31


### Read pretrained model

In [16]:
import tensorflow as tf

In [8]:
SkyValidator.training_params.get('job_dir')

'gs://geo-ai/Models/0/1581938271/'

In [43]:
job_dir = SkyValidator.training_params.get('job_dir')
model_dir = job_dir + 'model/'

In [ ]:
tf.saved_model.load(model_dir)

In [44]:
model = tf.contrib.keras.models.load_model(model_dir)

ValueError: Importing a SavedModel with tf.saved_model.load requires a 'tags=' argument if there is more than one MetaGraph. Got 'tags=None', but there are 3 MetaGraphs in the SavedModel with tag sets [['train'], ['eval'], ['serve']]. Pass a 'tags=' argument to load this SavedModel.

***
## **Prediction of Deep Learning models**
**Create `Predictor` object**

In [ ]:
SkyPredictor = skydl.Predictor()

**Get token**

In [ ]:
SkyPredictor.get_token(email='iker.sanchez@vizzuality.com')

### Select model

In [ ]:
SkyPredictor.models

In [ ]:
versions = SkyPredictor.select_model(model_name='water_quality')

### Select version

In [ ]:
SkyPredictor.select_version(version=versions[0])

## Creation of Geostore

We select the areas in which we will perform the prediction.

In [ ]:
atts={'geojson': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {},
    'geometry': {'type': 'Polygon',
     'coordinates': [[[0.5548095703125, 40.496048060627885],
                      [0.9558105468749999,40.496048060627885],
                      [0.9558105468749999,40.83667117059108],
                      [0.5548095703125,40.83667117059108],
                      [0.5548095703125,40.496048060627885]]]}}]}}

In [ ]:
SkyPredictor.create_geostore_from_geojson(atts)

## Predicting in AI Platform

In [ ]:
SkyPredictor.predict_ai_platform()